# Explore Data Extracted

Quick guide on how to use the functions and classes of this Repo. Alternatively you can simply run 

`python ./src/clinical_trials_gov_data/data_extraction/build_data_sources.py`

1. Extract data using the Clinical Trials Gov API 
2. Create DB and tables and extract protocols.
3. Brief data exploration

In [ ]:
# imports
import json

import pandas as pd

from clinical_trials_data.data_extraction.data_api import execute_curl_and_save_response
from clinical_trials_data.data_extraction.create_sqlite import (
    build_db_cursor,
    create_study_database,
    build_db_tuples,
    build_processed_studies_set,
)

from clinical_trials_data.data_extraction.extract_protocol_text import (
    extract_pdfs_parallel,
)


## 1. Connect to CTG API and extract data

In [ ]:
# Curl command as required by the CTG API. For more info visit https://clinicaltrials.gov/data-api/
# This is a basic query of completed studies, but the API allows for much more filtering options. 
curl_command = '''curl -X GET "https://clinicaltrials.gov/api/v2/studies?format=json&markupFormat=markdown&filter.overallStatus=COMPLETED&pageSize=20" -H "accept: application/json"'''
filename = "./data/raw/ctg-studies.json"
# 
data = execute_curl_and_save_response(curl_command, filename, return_data=True)["studies"]

## 2. Create DataBase and extract protocol text

In [ ]:
db_file_path = "../data/processed/study_protocols.db"
protocol_text_json_path = "../data/processed/procotol_text_json.json"

N = 20  # sample of studies -> this can be removed by I leave for testing

print("creating Data Base")
# build db connection
db_cursor, db_connection = build_db_cursor(db_file_path)
# Create the DB
create_study_database(db_cursor)
print("DB created")
# Get the data in the right format
# Extract protocol text requires connection to the internet
# Warning -> Remove the [: N] if you want to run in the whole set
processed_studies = build_processed_studies_set(
    data[:N], extract_protocol_text=True
)
# populate the tuples
studies_sql_tuples, study_design_sql_tuples = build_db_tuples(processed_studies)
print(f"Adding data to DB at {db_file_path}")
# finally, add the data
# TODO -> create a function for this??
db_cursor.executemany(
    """
    REPLACE INTO Studies (
        study_id,
        conditions,
        short_description,
        long_description,
        protocol_url,
        status
    )
    VALUES (?, ?, ?, ?, ?, ?)
""",
    studies_sql_tuples,
)
# adding values to study_design table
db_cursor.executemany(
    """
    REPLACE INTO Study_design (
        study_id,
        study_type,
        phases,
        allocation,
        enrollment_target
    )
    VALUES (?, ?, ?, ?, ?)
""",
    study_design_sql_tuples,
)

db_connection.commit()
db_connection.close()
print("Data Added into DB")

# Now get the protocol text from the internet
print("Extracting texts from protocol's urls")
procotol_txt_json = extract_pdfs_parallel(processed_studies)
# Save the output json
with open(protocol_text_json_path, "w") as f:
    json.dump(procotol_txt_json, f, indent=4)
print(f"Protocol Text Json File saved at {protocol_text_json_path}")

## 3. Explore the data

In [ ]:

# Connect to the SQLite database
db_cursor, db_connection = build_db_cursor(db_file_path)


# Query the database tables
query = "SELECT * FROM studies"
df_studies = pd.read_sql_query(query, db_connection)


query = "SELECT * FROM study_design"
df_design = pd.read_sql_query(query, db_connection)

# Close the database connection
db_connection.close()



In [22]:
df_studies.head()

,study_id,conditions,short_description,long_description,protocol_url,status
0,NCT00021918,Cardiovascular Diseases | Atherosclerosis | He...,\nTo examine the independent association of se...,\nTo examine the independent association of se...,,COMPLETED
1,NCT00093418,Adult Acute Megakaryoblastic Leukemia (M7) | A...,\nThis randomized phase II trial is studying 4...,\nThis randomized phase II trial is studying 4...,,COMPLETED
2,NCT00606918,Amyotrophic Lateral Sclerosis & Other Neuromus...,\nThere is a great need for the development of...,\nThere is a great need for the development of...,,COMPLETED
3,NCT00629018,Dilated Cardiomyopathy,\nSeveral studies have documented that transpl...,\nSeveral studies have documented that transpl...,,COMPLETED
4,NCT01054118,"Diabetes Mellitus, Type 2","\nThis study will assess the safety, tolerabil...","\nThis study will assess the safety, tolerabil...",,COMPLETED


In [23]:
df_design.head()

,id,study_id,study_type,phases,allocation,enrollment_target
0,1,NCT03104218,INTERVENTIONAL,NA,RANDOMIZED,40.0
1,2,NCT00093418,INTERVENTIONAL,PHASE2,RANDOMIZED,296.0
2,3,NCT04414618,INTERVENTIONAL,PHASE2,RANDOMIZED,42.0
3,4,NCT06034418,INTERVENTIONAL,NA,RANDOMIZED,28.0
4,5,NCT01054118,INTERVENTIONAL,PHASE1,RANDOMIZED,18.0


In [24]:
procotol_txt_json["NCT01951118"]

{'conditions': ['Alzheimer Disease',
  'Mild Cognitive Impairment',
  'Delirium, Dementia, Amnestic, Cognitive Disorders'],
 'description': "\nOlfactory identification deficits occur in patients with Alzheimer's disease (AD), are associated with disease severity, predict conversion from mild cognitive impairment (MCI) to AD and are associated with healthy elderly subjects developing MCI. Odor (olfactory) identification deficits may reflect degeneration of cholinergic inputs to the olfactory bulb and other olfactory brain regions. Acetylcholinesterase inhibitors (ACheI) like donepezil show modest effects in improving cognition but can be associated with adverse effects and increased burden and costs because of the need for prolonged, often lifelong, treatment. Converging findings on odor identification test performance (UPSIT, scratch and sniff 40-item test) from four pilot studies, including two of our own, suggest that acute change in the UPSIT in response to an anticholinergic challe